In [1]:
import pandas as pd
import numpy as np

# Loading .csv dataset
INPUT_PATH = r"C:\Users\Mary\D6_Retail-Sales-Dataset\data\raw\retail_sales_dataset.csv"
df_raw = pd.read_csv(INPUT_PATH)

# Keeping original copy for before and after comparison
df = df_raw.copy()

# Adding functions
def clean_column_names(columns):
    """Change column names to snake_case and remove extra spaces"""
    return (pd.Index(columns)
    .str.strip()
    .str.lower()
    .str.replace(r"[^\w\s]", "", regex=True)
    .str.replace(r"\s+", "_", regex=True))
def standardize_text(series):
    """Remove spaces from text"""
    return series.astype(str).str.strip()
def summarize_df(df_in, label="SUMMARY"):
    """Print a dataset summary: shape, dtypes, missing values, duplicates, quick stats for columns """
    print("\n" + "=" * 70)
    print(f"{label}")
    print("=" * 70)
    print(f"Rows, Columns: {df_in.shape[0]}, {df_in.shape[1]}")
    print(f"Duplicate rows (full row): {df_in.duplicated().sum()}")

    # Missing values
    missing = df_in.isna().sum().sort_values(ascending=False)
    missing = missing[missing > 0]
    print("\nMissing values (only columns with >0 missing):")
    if len(missing) == 0:print("None")
    else:print(missing)

    # Dtypes
    print("\nColumn dtypes:")
    print(df_in.dtypes)

    # Checking columns where applicable
    if "date" in df_in.columns:
        dt_min = df_in["date"].min() if pd.api.types.is_datetime64_any_dtype(df_in["date"]) else "not datetime yet"
        dt_max = df_in["date"].max() if pd.api.types.is_datetime64_any_dtype(df_in["date"]) else "not datetime yet"
        print(f"\nDate range: {dt_min}  ->  {dt_max}")
    for c in ["age", "quantity", "price_per_unit", "total_amount"]:
        if c in df_in.columns:
            print(f"\n{c} stats:")
            print(df_in[c].describe())
    for c in ["gender", "product_category"]:
        if c in df_in.columns:
            print(f"\nTop values for {c}:")
            print(df_in[c].value_counts(dropna=False).head(10))

# Creating summary of original file for before and after comparison
summarize_df(df, label="BEFORE CLEANING")

# Keeping track of before and after cleaning metrics
report = {"start_rows": len(df),"start_cols": df.shape[1]}

# Cleaning column names
old_cols = df.columns.tolist()
df.columns = clean_column_names(df.columns)
new_cols = df.columns.tolist()
report["renamed_columns"] = sum([1 for o, n in zip(old_cols, new_cols) if o != n])

# Standardizing text columns
text_cols = ["customer_id", "gender", "product_category"]
for col in text_cols:
    if col in df.columns:df[col] = standardize_text(df[col])

# Standardizing product category formatting
if "product_category" in df.columns:
    before_cat_unique = df["product_category"].nunique(dropna=False)
    df["product_category"] = df["product_category"].str.title()
    after_cat_unique = df["product_category"].nunique(dropna=False)
    report["category_unique_before"] = before_cat_unique
    report["category_unique_after"] = after_cat_unique

# Adjusting data type date to datetime
if "date" in df.columns:
    before_bad_dates = df["date"].isna().sum()  # still string, but count NA now
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    after_bad_dates = df["date"].isna().sum()
    report["date_parse_na_before"] = before_bad_dates
    report["date_parse_na_after"] = after_bad_dates

# Number conversions if applicable
numeric_cols = ["age", "quantity", "price_per_unit", "total_amount"]
for col in numeric_cols:
    if col in df.columns:
        before_non_numeric_na = df[col].isna().sum()
        df[col] = pd.to_numeric(df[col], errors="coerce")
        after_non_numeric_na = df[col].isna().sum()
        report[f"{col}_na_before_numeric_convert"] = before_non_numeric_na
        report[f"{col}_na_after_numeric_convert"] = after_non_numeric_na

# Checking for missing values
critical_cols = ["transaction_id","date","customer_id","product_category","quantity","price_per_unit"]
present_critical = [c for c in critical_cols if c in df.columns]
rows_before_dropna = len(df)
df = df.dropna(subset=present_critical)
report["rows_dropped_missing_critical"] = rows_before_dropna - len(df)

# Fill in any missing ages with median value
if "age" in df.columns:
    age_na_before = df["age"].isna().sum()
    df["age"] = df["age"].fillna(df["age"].median())
    age_na_after = df["age"].isna().sum()
    report["age_filled_n"] = age_na_before - age_na_after

# Removing any duplicates
rows_before_dupes = len(df)
if "transaction_id" in df.columns:df = df.drop_duplicates(subset=["transaction_id"])
else:df = df.drop_duplicates()
report["rows_removed_duplicates"] = rows_before_dupes - len(df)

# Removing any necessary data points and tracking for before and after
rows_before = len(df)
if "age" in df.columns:
    df = df[df["age"].between(12, 100)]
report["rows_removed_bad_age"] = rows_before - len(df)
rows_before = len(df)
if "quantity" in df.columns:
    df = df[df["quantity"] > 0]
report["rows_removed_bad_quantity"] = rows_before - len(df)
rows_before = len(df)
if "price_per_unit" in df.columns:
    df = df[df["price_per_unit"] > 0]
report["rows_removed_bad_price_per_unit"] = rows_before - len(df)

# Fixing total amount calculation
if set(["quantity", "price_per_unit", "total_amount"]).issubset(df.columns):
    mismatches_before = (df["total_amount"] != (df["quantity"] * df["price_per_unit"])).sum()
    df["total_amount"] = df["quantity"] * df["price_per_unit"]
    mismatches_after = (df["total_amount"] != (df["quantity"] * df["price_per_unit"])).sum()
    report["total_amount_mismatches_before_fix"] = int(mismatches_before)
    report["total_amount_mismatches_after_fix"] = int(mismatches_after)

# Prep Cleaning for visualization work
if "date" in df.columns:
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["month_name"] = df["date"].dt.strftime("%b")
    df["weekday"] = df["date"].dt.day_name()
for col in ["gender", "product_category", "month_name", "weekday"]:
    if col in df.columns:df[col] = df[col].astype("category")

# Creating after summary of cleaned data
summarize_df(df, label="AFTER CLEANING")

# Printing cleaning report with the before and after metrics
report["final_rows"] = len(df)
report["final_cols"] = df.shape[1]
report["rows_removed_total"] = report["start_rows"] - report["final_rows"]
print("\n" + "=" * 70)
print("CLEANING REPORT (WHAT CHANGED)")
print("=" * 70)
for k, v in report.items():print(f"{k}: {v}")

# Saving cleaned data
OUTPUT_PATH = r"C:\Users\Mary\D6_Retail-Sales-Dataset\data\processed\retail_sales_dataset_cleaned.csv"
df.to_csv(OUTPUT_PATH, index=False)
print("\nCleaned file saved to:")
print(OUTPUT_PATH)


BEFORE CLEANING
Rows, Columns: 1000, 9
Duplicate rows (full row): 0

Missing values (only columns with >0 missing):
None

Column dtypes:
Transaction ID       int64
Date                object
Customer ID         object
Gender              object
Age                  int64
Product Category    object
Quantity             int64
Price per Unit       int64
Total Amount         int64
dtype: object

AFTER CLEANING
Rows, Columns: 1000, 13
Duplicate rows (full row): 0

Missing values (only columns with >0 missing):
None

Column dtypes:
transaction_id               int64
date                datetime64[ns]
customer_id                 object
gender                    category
age                          int64
product_category          category
quantity                     int64
price_per_unit               int64
total_amount                 int64
year                         int32
month                        int32
month_name                category
weekday                   category
dtype: objec